In [4]:
from urllib.parse import urlencode
import requests
from pyquery import PyQuery as pq

base_url="https://m.weibo.cn/api/container/getIndex?"
headers={
    'Host':'m.weibo.cn',
    'Referer':'https://m.weibo.cn/u/3197187502',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15',
    'X-Requested-With':'XMLHttpRequest'
}
def get_page(page):
    params={
        'type':'uid',
        'value':'6177861155',
        'containerid':'1076033197187502',
        'page':page
    }
    url=base_url+urlencode(params)
    try:
        response=requests.get(url,headers=headers)
        if response.status_code==200:
            return response.json()
    except requests.ConnectionError as e:
        print('Error',e.args)
        
def parse_page(json):
    if json:
        items=json.get('data').get('cards')
        for item in items:
            item=item.get('mblog')
            print(item)
            if item is None:
                continue
            weibo={}
            weibo['date']=item.get('created_at')
            weibo['id']=item.get('id')
            weibo['text']=pq(item.get('text')).text()
            weibo['source']=item.get('source')
            weibo['attitudes']=item.get('attitudes_count')
            weibo['comments']=item.get('comments_count')
            weibo['reposts']=item.get('reposts_count')
            weibo['pictures']=item.get('original_pic')
            
            yield weibo
        break
            
        
if __name__ =='__main__':
    for i in range(1,100):
        json=get_page(i)
        results=parse_page(json)
        for result in results:
            with open ('/Users/limuyle/Desktop/weibo.txt','a',encoding='utf-8') as f:
                f.write(str('date: '+result['date']+'\n'+'text: '+result['text'].replace('\n','\t')+'\n'+'source: '+result['source']+'\t'+'attitudes: '+str(result['attitudes'])+'\t'+'comments: '+str(result['comments'])+'\t'+'reposts: '+str(result['reposts'])))
                f.write('\n')
    print('OK')

OK


In [ ]:
from wordcloud import WordCloud,ImageColorGenerator 
import jieba        #分词库
import imageio
import numpy as np    #科学数学计算包
import matplotlib.pyplot as plt   #数学绘图库
from PIL import Image
import re

# color_mask=imageio.imread('/Users/limuyle/Desktop/timg.jpg')
def get_wordcloud(imagepath,filepath,savepath):
    with open(filepath) as f:
#         读取文本
        txt=f.read()
#         分词
        cut_txt=jieba.cut(txt,cut_all=False)
        result="/".join(cut_txt)
#         制作背景图
#         image=Image.open(imagepath)
#         graph=np.array(image)
        pic=imageio.imread(imagepath)

        cloud=WordCloud(font_path='/Users/limuyle/Downloads/simhei.ttf',background_color='white',mask=pic,width=1000,height=500,max_font_size=50,max_words=2000)
        wordcloud=cloud.generate(result)
        
#         重新绘色
       
#         image_color=ImageColorGenerator(graph)
#         wordcloud.recolor(color_func=image_color)
    
#        保存并展示图片
        wordcloud.to_file(savepath)
        plt.figure('wordcloud')
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.show()
        
if __name__ == '__main__':
#     背景图路径，文本路径，保存文件路径
    get_wordcloud('/Users/limuyle/Desktop/wordcloud/yaya/redheart.jpg','/Users/limuyle/Desktop/wordcloud/yaya/bilibilicomments.txt','/Users/limuyle/Desktop/wordcloud/yaya/wordcloud.jpg')
        
        
        

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [ ]:
from urllib.parse import urlencode
import requests
from pyquery import PyQuery as pq
import os
from hashlib import md5
from multiprocessing.pool import Pool


base_url="https://m.weibo.cn/api/container/getIndex?"
headers={
    'Host':'m.weibo.cn',
    'Referer':'https://m.weibo.cn/u/3197187502',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15',
    'X-Requested-With':'XMLHttpRequest'
}
def get_page(page):
    params={
        'type':'uid',
        'value':'6177861155',
        'containerid':'1076033197187502',
        'page':page
    }
    url=base_url+urlencode(params)
    try:
        response=requests.get(url,headers=headers)
        if response.status_code==200:
            return response.json()
    except requests.ConnectionError as e:
        print('Error',e.args)
        
def parse_page(json):
    if json:
        items=json.get('data').get('cards')
        for item in items:
            
            item=item.get('mblog')
#             print('=====================')
#             print(item)
            if item is None:
                continue
            pics=item.get('pics')
            if pics is None:
                continue
            title=item.get('created_at')
            
            for pic in pics:
                large=pic.get('large')
                url=large.get('url')
                yield{
                    'image':url,
                    'title':title
                }
#             print('=====================')
            
            
#             yield weibo
           
#         user-page_info-pics

def save_image(item):
    if not os.path.exists('/Users/limuyle/Desktop/weibopics/'+item.get('title')):
        os.mkdir('/Users/limuyle/Desktop/weibopics/'+item.get('title'))
    try:
        response = requests.get(item.get('image'))
        if response.status_code == 200:
            file_path = '{0}/{1}.{2}'.format('/Users/limuyle/Desktop/weibopics/'+item.get('title'), md5(response.content).hexdigest(), 'jpg')
            if not os.path.exists(file_path):
                with open(file_path, 'wb')as f:
                    f.write(response.content)
            else:
                print('Already Downloaded', file_path)
    except requests.ConnectionError:
        print('Failed to save image')
        
def main(page):
    json=get_page(page)
    for item in parse_page(json):
        save_image(item)
    
        
if __name__ =='__main__':
    pool=Pool()
    groups=([x for x in range(1,101)])
    pool.map(main,groups)
    pool.close()
    pool.join()
    print('OK')
    